In [ ]:
# import relevant libraries
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from IPython.display import display, Markdown
import calendar
%config InlineBackend.figure_format = 'retina'

# load data into the dataframe
# IMPORTANT! note the conversions.csv file was too large to be uploaded to GitHub, even when the full dataset was reduced to releavnt rows and columns
# and also once zipped. Therefore if you want to run this code fully, then you will need to download `conversions.csv` from the following URL 
# directly from Kaggle https://www.kaggle.com/datasets/anandaramg/unsplash-image-download-data/data
colours_df = pd.read_csv('colors.csv')
print('downloaded colours: %s' % colours_df.shape[0])
conversions_df = pd.read_csv('conversions.csv')
print('downloaded conversions: %s' % conversions_df.shape[0])
photos_df = pd.read_csv('photos.csv')
print('downloaded photos: %s' % photos_df.shape[0])
# data downloaded from https://www.kaggle.com/datasets/anandaramg/unsplash-image-download-data/data (November 2024)
